<a href="https://colab.research.google.com/github/namwootree/Certificate/blob/main/Big_Data_Analytics/1_%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_%EA%B8%B0%EC%82%AC_%EC%8B%A4%EA%B8%B0_%EB%8C%80%EB%B9%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 유형

In [1]:
import pandas as pd
import numpy as np

In [75]:
df1 = pd.read_excel('/content/빅데이터분석기사_실기_데이터.xlsx')

In [69]:
print(df1.shape) # row 36883 column 14

(36833, 14)


In [70]:
print(df1.columns)

Index(['날짜', 'SKU ID', 'SKU 명', '바코드', '발주가능상태', '입고수량', '쿠팡출고수량', '쿠팡현재재고수량',
       '자재번호', '자재명', '카테고리', '입수', '출고수량', '재고수량'],
      dtype='object')


In [71]:
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36833 entries, 0 to 36832
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   날짜        36833 non-null  object 
 1   SKU ID    36833 non-null  int64  
 2   SKU 명     36833 non-null  object 
 3   바코드       36833 non-null  object 
 4   발주가능상태    36833 non-null  object 
 5   입고수량      36833 non-null  int64  
 6   쿠팡출고수량    36833 non-null  int64  
 7   쿠팡현재재고수량  36833 non-null  int64  
 8   자재번호      36833 non-null  int64  
 9   자재명       36833 non-null  object 
 10  카테고리      36833 non-null  object 
 11  입수        36833 non-null  int64  
 12  출고수량      36833 non-null  float64
 13  재고수량      36833 non-null  float64
dtypes: float64(2), int64(6), object(6)
memory usage: 3.9+ MB
None


출고수량 항목에 데이터에서 출고수향이 10개 이상인 데이터를 추출하여 재고수량의 평균을 구하시오

In [72]:
cond1 = (df1['출고수량']>=10)

df2 = df1.loc[cond1]

# print(df2.shape) # 출고수량이 10개 이상인 데이터 개수 : 6276

print(df2['재고수량'].mean()) # 144.994

144.99403119936906


재고수량 데이터를 standard scaling을 실시하여 0.5이상의 데이터의 개수를 구하라

In [73]:
# standard scaling : 평균 0 표준편차 1
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

scaler = StandardScaler()
# print(scaler)

scaler.fit(df1[['재고수량']])
result = scaler.transform(df1[['재고수량']])
df_result = pd.DataFrame(result)

# print(df_result.describe()) # mean 0 std 1
# print(df_result.shape) # row 36883 col 1

cond1 = (df_result[0]>0.5)

df_result2 = df_result.loc[cond1]

print(df_result2.shape) # 0.5 이상인 데이터 개수 1791

(1791, 1)


입고수량 데이터에서 가장 큰 상위 10개 데이터를 가장 작은 최소값으로 변환한 뒤, 평균을 계산

In [78]:
df1_top_10 = df1.sort_values(by='입고수량', ascending=False).head(10)

# print(df1_top_10.shape)
# print(df1_top_10['입고수량'].describe()) # min : 7287.000000

cond1 = (df1['입고수량']>=df1_top_10['입고수량'].min())

df1.loc[cond1, '입고수량(clean)'] = df1['입고수량'].min()
df1.loc[~cond1, '입고수량(clean)'] = df1['입고수량']

# print(df1[['입고수량', '입고수량(clean)']].describe())

print(df1['입고수량(clean)'].mean()) # 53.594

53.59441261911873


데이터에서 학습 데이터로 70%를 추출하여 출고수량 데이터의 표준편차와 검증 데이터의 출고수량 표준편차 차이 계산 (random_state=1234)


In [81]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df1, test_size=0.3, random_state=1234)

# print(df_train.shape) # 25783
# print(df_test.shape) # 11050

print(df_train['출고수량'].std() - df_test['출고수량'].std()) # -0.361

-0.361077688474964


입수의 이상치를 모두 합한 값을 계산하라
(이상치 : 3사분위수 + (IQR * 1.5배))

In [86]:
# IQR : (25% ~ 75%)
# print(df1['입수'].describe()) # IQR : 6
# print(6 * 1.5) # 9

cond1 = (df1['입수']>12+9)
df_out = df1.loc[cond1]
# print(df_out.shape) # 2102

print(df_out['입수'].sum()) # 56795

56795


출고수량의 이상치를 모두 합한 값을 구하시오 (이상치 : 표준편차 3배)

In [91]:
# print(df1['출고수량'].describe()) # std : 19.469565
# print(df1['출고수량'].std()*3) # 58.4086951362902

cond1 = (df1['출고수량']>=(df1['출고수량'].std()*3))
df1_out = df1.loc[cond1]

# print(df1_out.shape) # 932

print(df1_out['출고수량'].sum()) # 92892.01

92892.0111111111


# 2 유형

재고수량이 500개 이상인 값을 1 / 미만을 0으로 변환하여 Target column을 생성한 뒤, 해당 target column에 대한 분류모델을 생성하고 평가하시오

In [92]:
cond1  = (df1['재고수량']>=500)

df1.loc[cond1, 'Target'] = 1
df1.loc[~cond1, 'Target'] = 0

# print(df1['Target'].value_counts())

0.0    32586
1.0     4247
Name: Target, dtype: int64


In [95]:
# x / y 분할
# x 범주형 데이터 더미화 
# 학습/검증 데이터 분할
# 파이프라인 구축 (특성 공학 / 결측치 처리 + 분류 학습)
# 교차검증 + hyper parmeter 튜닝
# 평가

# print(df1.columns)

# x / y 분할
y = df1['Target']
x = df1.drop(columns=['Target', '날짜', 'SKU ID', 'SKU 명', '바코드',
                      '자재번호', '자재명', '재고수량', '입고수량(clean)'])

# x 범주형 데이터 더미화 
x_dummy = pd.get_dummies(x)

In [96]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [97]:
x_train, x_test, y_train, y_test = train_test_split(x_dummy, y, 
                                                    test_size=0.3,
                                                    random_state=1234)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(25783, 16)
(11050, 16)
(25783,)
(11050,)


In [98]:
pipe_list = [('impute', KNNImputer()),
             ('model', RandomForestClassifier())]

pipe_model = Pipeline(pipe_list)

# print(pipe_model)

Pipeline(steps=[('impute', KNNImputer()), ('model', RandomForestClassifier())])


In [108]:
hyper_param = {
               'model__min_samples_leaf':range(5, 10),
               'model__class_weight':[None, 'balanced'],
               'model__n_estimators':[50, 100, 150],
              }

grid_model = GridSearchCV(pipe_model,
                          param_grid=hyper_param,
                          cv=3, n_jobs=-1, scoring='f1')

grid_model.fit(x_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('impute', KNNImputer()),
                                       ('model', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'model__class_weight': [None, 'balanced'],
                         'model__min_samples_leaf': range(5, 10),
                         'model__n_estimators': [50, 100, 150]},
             scoring='f1')

In [109]:
best_model = grid_model.best_estimator_
print(best_model)

Pipeline(steps=[('impute', KNNImputer()),
                ('model',
                 RandomForestClassifier(min_samples_leaf=5, n_estimators=150))])


In [110]:
y_train_pred = best_model.predict(x_train)
y_test_pred = best_model.predict(x_test)

print(y_train_pred.shape)
print(y_test_pred.shape)

(25783,)
(11050,)


In [111]:
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     22804
         1.0       1.00      0.97      0.98      2979

    accuracy                           1.00     25783
   macro avg       1.00      0.99      0.99     25783
weighted avg       1.00      1.00      1.00     25783

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      9782
         1.0       0.99      0.94      0.96      1268

    accuracy                           0.99     11050
   macro avg       0.99      0.97      0.98     11050
weighted avg       0.99      0.99      0.99     11050



In [112]:
result = pd.DataFrame(y_test_pred)
result.to_csv('202206016.csv')